In [4]:
import pandas as pd 
import numpy as np

import tensorflow as tf 
from tensorflow.keras import *
from tensorflow.keras.layers import *

df = pd.read_csv(
    './aws_a5_1_all.csv'
)

df.head()

,num,cputime_dif,depth,high,image,tension,time,velocity,volume,weld_velocity,width,wire_l
0,0,0.109213,-2.612515,2.251549,4482.0,20.0,2.614,5.5,0.704238,8.0,4.850269,15.0
1,1,0.113643,-2.568461,2.261362,4497.0,20.0,2.629,5.5,0.093257,8.0,5.014591,15.0
2,2,0.187733,-2.284407,2.307767,5361.0,20.0,3.493,5.5,5.371661,8.0,6.496406,15.0
3,3,0.124918,-2.072830,2.291701,5433.0,20.0,3.565,5.5,0.447639,8.0,7.534782,15.0
4,4,0.110063,-2.497762,2.195335,6325.0,20.0,4.457,5.5,5.545741,8.0,7.738949,15.0


In [8]:
features = np.array(df[['high' , 'tension' , 'velocity' , 'volume' , 'weld_velocity' , 'width' , 'wire_l']])
labels = np.array(df['depth'])

labels_mean = np.mean(labels)
labels_std = np.std(labels)


array([-2.61251488, -2.56846062, -2.28440721, ..., -3.09060336,
       -3.09060336, -3.09058904])

In [9]:
numeric_dataset = tf.data.Dataset.from_tensor_slices((features, labels))

for row in numeric_dataset.take(3):
    print(row)

(<tf.Tensor: shape=(7,), dtype=float64, numpy=
array([ 2.25154901, 20.        ,  5.5       ,  0.7042383 ,  8.        ,
        4.85026926, 15.        ])>, <tf.Tensor: shape=(), dtype=float64, numpy=-2.612514882967101>)
(<tf.Tensor: shape=(7,), dtype=float64, numpy=
array([ 2.26136213, 20.        ,  5.5       ,  0.09325717,  8.        ,
        5.01459088, 15.        ])>, <tf.Tensor: shape=(), dtype=float64, numpy=-2.5684606173793463>)
(<tf.Tensor: shape=(7,), dtype=float64, numpy=
array([ 2.30776663, 20.        ,  5.5       ,  5.37166119,  8.        ,
        6.49640607, 15.        ])>, <tf.Tensor: shape=(), dtype=float64, numpy=-2.284407206321464>)


In [10]:
mean = np.mean(features , axis=0)
mean

array([ 1.17014343, 28.27534791,  6.60735586,  0.83287074,  9.5387674 ,
        6.60738607, 13.73389776])

In [11]:
standard = np.std(features , axis= 0)
standard

array([0.66960875, 2.44882587, 0.99422066, 1.09952596, 1.94608954,
       1.11983017, 1.04891994])

In [21]:
features

array([[-2.61251488,  2.25154901, 20.        , ...,  8.        ,
         4.85026926, 15.        ],
       [-2.56846062,  2.26136213, 20.        , ...,  8.        ,
         5.01459088, 15.        ],
       [-2.28440721,  2.30776663, 20.        , ...,  8.        ,
         6.49640607, 15.        ],
       ...,
       [-3.09060336,  0.05518935, 29.        , ..., 12.        ,
         5.40717952, 13.6       ],
       [-3.09060336,  0.05518935, 29.        , ..., 12.        ,
         5.40717952, 15.        ],
       [-3.09058904,  0.05518909, 29.        , ..., 12.        ,
         5.40714196, 15.        ]])

In [12]:
def normalize(data , label) :
    return (data - mean) / standard , (label - labels_mean) / labels_std

x_test = numeric_dataset.take(10)

numeric_batches = numeric_dataset.shuffle(1000).batch(4)
numeric_batches = numeric_batches.map(normalize)

In [36]:
def get_basic_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
      tf.keras.layers.Dense(5, activation='relu'),
      tf.keras.layers.Dense(5, activation='relu'),
    tf.keras.layers.Dense(1 , activation='tanh')
  ])

  model.compile(optimizer= optimizers.Adam() ,
                loss= losses.mean_squared_error ,
                metrics= [metrics.MeanSquaredError()])
  return model

model = get_basic_model()

model.fit(numeric_batches , batch_size=4 , epochs=30)

Epoch 1/30

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

252/252 [==============================] - 2s 10ms/step - loss: 0.8276 - mean_squared_error: 0.8276
Epoch 2/30
252/252 [==============================] - 3s 10ms/step - loss: 0.4477 - mean_squared_error: 0.4477
Epoch 3/30
252/252 [==============================] - 3s 11ms/step - loss: 0.2426 - mean_squared_error: 0.2426 0s - loss: 0.2517 - mean
Epoch 4/30
252/252 [==============================] - 3s 11ms/step - loss: 0.2077 - mean_squared_error: 0.2077
Epoch 5/30
252/252 [===================

In [28]:
test = (np.array([[2.25154901, 20.,5.5,0.7042383 ,8.,4.85026926, 15.]]) - mean) / standard
print(test)
model.predict(test)[0][0]*labels_std + labels_mean

[[ 1.61498125 -3.37931252 -1.11379285 -0.116989   -0.79069712 -1.56909222
   1.20705326]]


-2.486492402942026

In [29]:
#important
model.save('./my_model')
x_test = x_test.map(normalize)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./my_model\assets


In [37]:
test

array([[ 1.61498125, -3.37931252, -1.11379285, -0.116989  , -0.79069712,
        -1.56909222,  1.20705326]])

In [ ]:
new_model = tf.keras.models.load_model('./my_model')

test = (np.array([[2.25154901, 20.,5.5,0.7042383 ,8.,4.85026926, 15.]]) - mean) / standard
print(test)
model.predict(test)[0][0]*labels_std + labels_mean

In [30]:
new_model = tf.keras.models.load_model('./my_model')

test = (np.array([[2.25154901, 20.,5.5,0.7042383 ,8.,4.85026926, 15.]]) - mean) / standard
print(test)
model.predict(test)[0][0]*labels_std + labels_mean


[[ 1.61498125 -3.37931252 -1.11379285 -0.116989   -0.79069712 -1.56909222
   1.20705326]]


-2.486492402942026

In [32]:
x , y = [] , []

for u , v in x_test.as_numpy_iterator() :
    x.append(u)
    y.append(v)

x = np.array(x)
y = np.array(y)

In [33]:
model.evaluate(x ,y)

1/1 [==============================] - 0s 6ms/step - loss: 0.1023 - mean_squared_error: 0.1023


[0.10226745903491974, 0.10226745903491974]

In [43]:
def predictwelddepth(high , tension , velocity, volume , weld_velocity , width , wire_l) :
    test = (np.array([[high, tension , velocity , volume , weld_velocity , width, wire_l]]) - mean) / standard
    res = new_model.predict(test)[0][0]*labels_std + labels_mean
    print("Weld bead depth = " + str(res)+ ' mm')

In [44]:
predictwelddepth(2.25154901, 20.,5.5,0.7042383 ,8.,4.85026926, 15.)

Weld bead depth = -2.486492402942026 mm
